In [39]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## 1.读取数据

In [40]:
data_path = "./data/gouxionghui_data/mojitainqi_8/data.xlsx"
data = pd.read_excel(data_path, na_values = np.nan)

## 2.了解数据、数据质量检查

In [41]:
data.head()

,城市,地区,城市AQI,PM2.5浓度,PM10浓度,首要污染物,污染等级,地区AQI
0,鞍山实时空气质量指数,明达新区,177,125μg/m³,228μg/m³,PM2.5,轻度污染,165
1,鞍山实时空气质量指数,千山,177,117μg/m³,145μg/m³,PM2.5,轻度污染,153
2,鞍山实时空气质量指数,深沟寺,177,138μg/m³,244μg/m³,PM2.5,轻度污染,183
3,鞍山实时空气质量指数,太平,177,126μg/m³,239μg/m³,PM2.5,轻度污染,166
4,鞍山实时空气质量指数,太阳城,177,142μg/m³,242μg/m³,PM2.5,轻度污染,189


In [42]:
print('数据维度:\n', '------------\n', data.shape)
print('*' * 20)
print('数据变量类型：\n', '------------\n',data.dtypes)

数据维度:
 ------------
 (1453, 8)
********************
数据变量类型：
 ------------
 城市         object
地区         object
城市AQI       int64
PM2.5浓度    object
PM10浓度     object
首要污染物      object
污染等级       object
地区AQI       int64
dtype: object


## 3.数据预处理

### 3.1 将`PM2.5浓度`，`PM10浓度`转换为数值型变量

In [43]:
data['PM2.5浓度值'] = data['PM2.5浓度'].str.split('μ').str.get(0)
# data['PM2.5浓度值'] = data['PM2.5浓度'].str.replace('μg/m³', '')
# data['PM2.5浓度单位'] = 'μ' + data['PM2.5浓度'].str.split('μ').str.get(1)
data.drop(['PM2.5浓度'], axis = 1, inplace = True)

data['PM10浓度值'] = data['PM10浓度'].str.split('μ').str.get(0)
# data['PM10浓度值'] = data['PM10浓度'].str.replace('μg/m³', '')
# data['PM10浓度单位'] = 'μ' + data['PM10浓度'].str.split('μ').str.get(1)
data.drop(['PM10浓度'], axis = 1, inplace = True)

In [44]:
data.loc[data['PM10浓度值'].str.contains('—'), 'PM10浓度值'] = np.nan
data.loc[data['首要污染物'] == '—', '首要污染物'] = np.nan

In [45]:
data['PM2.5浓度值'] = pd.to_numeric(data['PM2.5浓度值'], errors = 'coerce')
data['PM10浓度值'] = pd.to_numeric(data['PM10浓度值'], errors = 'coerce').astype('Int64')

### 2.2 类别型数据处理

* 将object类型转化为category类型
* 将有序category转换为排序的category

#### 2.2.1 `城市`，`地区`转换为无序类别型变量

In [46]:
data['城市'] = data['城市'].str.replace('实时空气质量指数', '')
data['城市'] = data['城市'].astype('category')
data['地区'] = data['地区'].astype('category')
data['首要污染物'] = data['首要污染物'].astype('category')

print(data['城市'].cat.categories)
print(data['地区'].cat.categories)
print(data['首要污染物'].cat.categories)

Index(['七台河', '三亚', '三明', '三门峡', '上海', '上饶', '东莞', '东营', '中卫', '中山',
       ...
       '黄冈', '黄南州', '黄山', '黄石', '黑河', '黔东南州', '黔南州', '黔西南州', '齐齐哈尔', '龙岩'],
      dtype='object', length=365)
Index(['1号', '22中南校区', '2号', '2号子站', 'JN子站', '一中', '一商场', '丁字口', '七星岩子站',
       '七贤岭',
       ...
       '龙岩师专', '龙新', '龙泉镇', '龙洲湾', '龙港区', '龙湖子站', '龙湾', '龙虎山', '龙赛医院',
       '龙隐路小学'],
      dtype='object', length=1264)
Index(['O3', 'PM10', 'PM2.5'], dtype='object')


#### 2.2.2 `污染等级`转换为有序类别型数据

In [47]:
levels = {
    '严重污染': 5,
    '中度污染': 4,
    '轻度污染': 3,
    '良': 2,
    '优': 1
}
data['污染等级'] = data['污染等级'] \
    .astype(CategoricalDtype(categories = ['优', '良', '轻度污染', '中度污染', '严重污染'], ordered = True)) \
    .cat.rename_categories(levels)
print(data.污染等级.cat.categories)
print(data.污染等级.cat.ordered)

Int64Index([1, 2, 3, 4, 5], dtype='int64')
True


In [48]:
data.dtypes

城市          category
地区          category
城市AQI          int64
首要污染物       category
污染等级        category
地区AQI          int64
PM2.5浓度值       int64
PM10浓度值        Int64
dtype: object

In [49]:
data.head()

,城市,地区,城市AQI,首要污染物,污染等级,地区AQI,PM2.5浓度值,PM10浓度值
0,鞍山,明达新区,177,PM2.5,3,165,125,228
1,鞍山,千山,177,PM2.5,3,153,117,145
2,鞍山,深沟寺,177,PM2.5,3,183,138,244
3,鞍山,太平,177,PM2.5,3,166,126,239
4,鞍山,太阳城,177,PM2.5,3,189,142,242
